### All these code runs on Kaggle's Notebook  

In [2]:
!pip install autogluon

In [8]:
import pandas as pd
df = pd.read_csv('../input/msbd5001-spring-2022/train.csv')
df

## try1

In [30]:
from autogluon.tabular import TabularDataset, TabularPredictor
predictor = TabularPredictor.load("AutogluonModels/ag-20220515_050224/")
predictor.leaderboard(train_data, silent=True)
y_pred = predictor.predict(test_data)
submission = y_pred.to_frame().reset_index().rename(columns = {'index':'id', 0:'label'})
submission.to_csv('test2.csv',index=False)

## try 2

In [45]:
from autogluon.tabular import TabularDataset, TabularPredictor
# TabularDataset是一个继承pandas.DataFrame的数据集类
train_data = TabularDataset('../input/msbd5001-spring-2022/train.csv').drop(columns="id")
test_data = TabularDataset('../input/msbd5001-spring-2022/test.csv').drop(columns="id")

#predictor = TabularPredictor(label='label', eval_metric='roc_auc').fit(train_data=train_data, presets = 'best_quality')
predictor = TabularPredictor(label='label', eval_metric='roc_auc').fit(train_data=train_data,
    num_bag_folds=10, num_bag_sets=5, num_stack_levels=3,
    hyperparameters = {'NN_TORCH': {'num_epochs': 10}, 'GBM': {'num_boost_round': 20}},  # last  argument is just for quick demo here, omit it in real applications
)
# 模型预测
y_pred = predictor.predict(test_data)

In [31]:
predictor = TabularPredictor.load("AutogluonModels/ag-20220515_052354/")
predictor.leaderboard(train_data, silent=True)

# Prediction

In [ ]:
#if you want to use my model saved in the zip, you could uncomment this
#predictor = TabularPredictor.load("AutogluonModels/ag-20220515_053929/")   #load the best model
y_pred = predictor.predict(test_data)

# Submision

In [47]:
#save test prediction result
submission = y_pred.to_frame().reset_index().rename(columns = {'index':'id', 0:'label'})
submission.to_csv('/kaggle/working/submission_2.csv',index = False)

In [39]:
# zip the files in the Output in Kaggle
import os
import zipfile
import datetime

def file2zip(packagePath, zipPath):
    '''
  :param packagePath: 文件夹路径
  :param zipPath: 压缩包路径
  :return:
  '''
    zip = zipfile.ZipFile(zipPath, 'w', zipfile.ZIP_DEFLATED)
    for path, dirNames, fileNames in os.walk(packagePath):
        fpath = path.replace(packagePath, '')
        for name in fileNames:
            fullName = os.path.join(path, name)
            name = fpath + '\\' + name
            zip.write(fullName, name)
    zip.close()


if __name__ == "__main__":
    # 文件夹路径
    packagePath = '/kaggle/working/'
    zipPath = '/kaggle/working/output.zip'
    if os.path.exists(zipPath):
        os.remove(zipPath)
    file2zip(packagePath, zipPath)
    print("打包完成")
    print(datetime.datetime.utcnow())


In [49]:
##download from the Output
import os
os.chdir('/kaggle/working/')
print(os.getcwd())
print(os.listdir("/kaggle/working/"))
from IPython.display import FileLink
##download the files in the output
#FileLink('output.zip')
FileLink('submission.csv')